In [104]:
import numpy as np 
import pandas as pd 
import os
import matplotlib.pyplot as plt

os.chdir("/Users/jacobrichards/Desktop/Personal_save/Python/Marketing_Prediction_Python_files")

data = pd.read_csv("data.csv", na_values=["", "NA"])

display(data.head())

,id,age,dist,income,gender,marital_status,target
0,1,73,4.371654,"90-99,999",M,S,1
1,2,89,1.582733,"100-149,999",M,NaN,1
2,3,85,1.223810,"10-19,999",F,S,1
3,4,76,2.962427,"90-99,999",M,M,1
4,5,76,2.594408,"10-19,999",M,S,1


In [105]:
import pandas as pd
import numpy as np

# Replace missing or empty values in columns 4, 5, 6 with "unknown"
data.iloc[:, [3, 4, 5]] = data.iloc[:, [3, 4, 5]].applymap(lambda x: "unknown" if pd.isna(x) or x == "" else x)

# Replace income with appropriate midpoints or custom values
income_mapping = {
    "unknown": 55000,              # replacing unknown with the median income level
    "Under $10k": 5000,            # midpoint of "Under $10k"
    "10-19,999": 15000,            # midpoint of "10-19,999"
    "20-29,999": 25000,            # midpoint of "20-29,999"
    "30-39,999": 35000,            # midpoint of "30-39,999"
    "40-49,999": 45000,            # midpoint of "40-49,999"
    "50-59,999": 55000,            # midpoint of "50-59,999"
    "60-69,999": 65000,            # midpoint of "60-69,999"
    "70-79,999": 75000,            # midpoint of "70-79,999"
    "80-89,999": 85000,            # midpoint of "80-89,999"
    "90-99,999": 95000,            # midpoint of "90-99,999"
    "100-149,999": 125000,         # midpoint of "100-149,999"
    "150 - 174,999": 162500,       # midpoint of "150 - 174,999"
    "175 - 199,999": 187500,       # midpoint of "175 - 199,999"
    "200 - 249,999": 225000,       # midpoint of "200 - 249,999"
    "250k+": 250000               # custom value for "250k+"
}
data["income"] = data["income"].map(income_mapping).astype(int)

# Replace gender with 1 for male, 0 for female, and 0 for unknown
gender_mapping = {"M": 1, "F": 0, "unknown": 0}
data["gender"] = data["gender"].map(gender_mapping).astype(int)

# Replace marital_status with 1 for married, 0 for single, and 1 for unknown
marital_status_mapping = {"M": 1, "S": 0, "unknown": 1}
data["marital_status"] = data["marital_status"].map(marital_status_mapping).astype(int)

# Convert target to categorical values (0 or 1)
data["target"] = data["target"].astype(int)

# Convert distance (dist) column to integer
data["dist"] = data["dist"].astype(int)

display(data.head())

/var/folders/2y/cbtx_pl154qczd091zb0d4z40000gn/T/ipykernel_35734/3234243408.py:5: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data.iloc[:, [3, 4, 5]] = data.iloc[:, [3, 4, 5]].applymap(lambda x: "unknown" if pd.isna(x) or x == "" else x)


,id,age,dist,income,gender,marital_status,target
0,1,73,4,95000,1,0,1
1,2,89,1,125000,1,1,1
2,3,85,1,15000,0,0,1
3,4,76,2,95000,1,1,1
4,5,76,2,15000,1,0,1


Standardize our variables 

In [106]:
from sklearn.preprocessing import StandardScaler

# Initialize the scaler
scaler = StandardScaler()

# Standardize the selected columns
data[['age', 'income', 'dist']] = scaler.fit_transform(data[['age', 'income', 'dist']])


display(data.head())

,id,age,dist,income,gender,marital_status,target
0,1,-0.445587,-0.576729,0.413826,1,0,1
1,2,1.541145,-1.018667,0.982723,1,1,1
2,3,1.044462,-1.018667,-1.103235,0,0,1
3,4,-0.073074,-0.871354,0.413826,1,1,1
4,5,-0.073074,-0.871354,-1.103235,1,0,1


build a logistic regression model

split data into test and train 

In [107]:
train, test = train_test_split(data, test_size=0.2, random_state=123, stratify=data["target"])

In [108]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

x_train = train[['age', 'dist', 'income','gender','marital_status']] #training predictor

y_train = train['target'] # training response 

x_test = test[['age', 'dist', 'income','gender','marital_status']] #test predictor

y_test = test[['target']] # test response 

model = LogisticRegression()
model.fit(x_train, y_train) # train the model on predictor and response from training data 

y_pred = model.predict(x_test) # evaluate model on test predictors for predicted binary outcome 

print(classification_report(y_test, y_pred)) # compair actual test response with predicted test response 

              precision    recall  f1-score   support

           0       0.76      0.99      0.86       600
           1       0.62      0.05      0.09       200

    accuracy                           0.76       800
   macro avg       0.69      0.52      0.48       800
weighted avg       0.72      0.76      0.67       800



ok well let's try with random forest to see what we can get 

In [110]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split

# Build Random Forest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=123)  # Adjust parameters as needed
rf_model.fit(x_train, y_train)

# Make predictions
y_pred = rf_model.predict(x_test)

# Evaluate performance
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.78      0.88      0.83       600
           1       0.41      0.26      0.31       200

    accuracy                           0.72       800
   macro avg       0.60      0.57      0.57       800
weighted avg       0.69      0.72      0.70       800

[[527  73]
 [149  51]]


In [111]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix

# Create KNN model
knn_model = KNeighborsClassifier(n_neighbors=5)  # K=5
knn_model.fit(x_train, y_train)

# Make predictions
y_pred = knn_model.predict(x_test)

# Evaluate the model
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.77      0.88      0.82       600
           1       0.39      0.23      0.29       200

    accuracy                           0.71       800
   macro avg       0.58      0.55      0.56       800
weighted avg       0.68      0.71      0.69       800

[[525  75]
 [153  47]]
